In [ ]:
import os
import requests
import time
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
# 노트북 최상단에 추가
import pandas as pd
pd.options.display.max_colwidth = 100
pd.options.display.unicode.east_asian_width = True


In [ ]:
from supabase import create_client, Client

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print("✅ 환경변수로 안전하게 연결!")

In [ ]:
# Supabase URL이 올바른지 확인
SUPABASE_URL  # 반드시 https:// 포함
SUPABASE_KEY 

print(f"접속 URL: {SUPABASE_URL}")  # URL 출력해서 확인

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [ ]:
from supabase import create_client, Client
import time
import socket
from supabase import create_client, Client
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Supabase 클라이언트 생성
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

def test_network():
    """네트워크 연결 테스트"""
    try:
        socket.create_connection(("8.8.8.8", 53), timeout=3)
        print("✓ 인터넷 연결 정상")
        return True
    except OSError as e:
        print(f"✗ 인터넷 연결 실패: {e}")
        return False

def test_supabase_connection(url, key):
    """Supabase 연결 테스트"""
    try:
        print(f"\n접속 시도: {url}")
        client = create_client(url, key)
        
        # 간단한 쿼리로 연결 테스트
        response = client.table("t_temp").select("*").limit(1).execute()
        print("✓ Supabase 연결 성공")
        return client
    except Exception as e:
        print(f"✗ Supabase 연결 실패: {e}")
        return None

# 1. 네트워크 테스트
if not test_network():
    print("\n인터넷 연결을 확인해주세요.")
    exit(1)

# 2. Supabase 연결 테스트
supabase = test_supabase_connection(SUPABASE_URL, SUPABASE_KEY)
if not supabase:
    print("\nSupabase URL과 KEY를 확인해주세요.")
    exit(1)

# 3. 데이터 수집 시작
print("\n=== 데이터 수집 시작 ===")
all_data = []
step = 1000
start = 0

while True:
    end = start + step - 1
    
    try:
        response = supabase.table("t_temp").select("*").range(start, end).execute()
        rows = response.data
        
        if not rows:
            print(f"\n총 {len(all_data)}건 수집 완료")
            break
        
        all_data.extend(rows)
        print(f"{start:,} ~ {end:,} 수집 완료, 누적 {len(all_data):,}건")
        
        start += step
        time.sleep(0.1)  # API 부하 방지
        
    except Exception as e:
        print(f"\n오류 발생 (위치: {start}): {e}")
        print("10초 후 재시도...")
        time.sleep(10)
        continue

print(f"\n최종 수집: {len(all_data):,}건")

In [ ]:
all_data = []
step = 1000  # 한 번에 가져올 row 수
start = 0

while True:
    end = start + step - 1
    response = supabase.table("t_temp").select("*").range(start, end).execute()
    
    rows = response.data
    if not rows:  # 더 이상 데이터 없으면 종료
        break

    all_data.extend(rows)
    print(f"{start} ~ {end} 수집 완료, 누적 {len(all_data)}건")

    start += step  # 다음 페이지로 이동

In [ ]:
len(all_data)

df = pd.DataFrame(all_data)

df

In [ ]:

# load_dotenv('../.env')
# API_KEY = os.getenv('PUBLIC_API_KEY')

# BASE_URL = f'http://openapi.seoul.go.kr:8088/{API_KEY}/json/tbLnOpendataRtmsV'

# years = [2023, 2024, 2025]
# districts = {
#     '서초구': '11650',
#     '강남구': '11680',
#     '송파구': '11710'
# }

# rows_all = []
# step = 1000

# for year in years:
#     for gu_nm, gu_cd in districts.items():
#         start = 1

#         while True:
#             end = start + step - 1
#             url = f'{BASE_URL}/{start}/{end}/{year}/{gu_cd}'

#             res = requests.get(url)
#             if res.status_code != 200:
#                 print('요청 실패:', year, gu_nm)
#                 break

#             data = res.json().get('tbLnOpendataRtmsV', {})
#             rows = data.get('row', [])

#             if not rows:
#                 break

#             rows_all.extend(rows)
#             print(f'{year} {gu_nm} {start}~{end} 수집')

#             start += step
#             time.sleep(2)

# df = pd.DataFrame(rows_all)

# if df.empty:
#     print("데이터가 없습니다.")
# else:
#     df = df[df['BLDG_USG'] == '아파트']

#     df = df[
#         [
#             'RCPT_YR',        # 접수연도
#             'CGG_CD',         # 자치구 코드
#             'CGG_NM',         # 자치구명
#             'STDG_CD',        # 법정동 코드
#             'STDG_NM',        # 법정동 명
#             'BLDG_NM',        # 건물명
#             'CTRT_DAY',       # 계약일
#             'THING_AMT',	  # 물건금액(만원)
#             'ARCH_AREA',	  # 건물면적(㎡)
#             'LAND_AREA',	  # 토지면적(㎡)
#             'FLR',	          # 층
#             'ARCH_YR',        # 건축년도
#             'BLDG_USG',       # 건물용도
#         ]
#     ]

#     # 계약일자 기준 필터링 추가
#     # CTRT_DAY를 datetime으로 변환
#     # 거래는 2022-12-31에 했으나 신고를 2023년에 하는 케이스가 존재
#     df['CTRT_DAY'] = pd.to_datetime(df['CTRT_DAY'], format='%Y%m%d', errors='coerce')
    
#     # 2023-01-01 ~ 2025-12-31 사이만 필터링
#     df = df[
#         (df['CTRT_DAY'] >= '2023-01-01') & 
#         (df['CTRT_DAY'] <= '2025-12-31')
#     ]

#     # 중복 제거
#     print(f"\n중복 제거 전: {len(df):,}건")
    
#     df_clean = df.drop_duplicates(subset=[
#         'CTRT_DAY',
#         'BLDG_NM', 
#         'ARCH_AREA',
#         'FLR',
#         'THING_AMT'
#     ], keep='first')

#     print(f"중복 제거 후: {len(df_clean):,}건")

#     # CTRT_DAY를 다시 문자열로 변환 (저장 시 형식 유지)
#     df_clean['CTRT_DAY'] = df_clean['CTRT_DAY'].dt.strftime('%Y%m%d')

#     df_clean.to_csv(
#         '../data/seoul_apartment_2023_2025_gangnam.csv',
#         index=False,
#         encoding='utf-8-sig'
#     )

#     display(df.head())

In [ ]:
# 저장된 csv 읽기
# df = pd.read_csv('../data/seoul_apartment_2023_2025_gangnam.csv', encoding='utf-8-sig')

In [ ]:
# 중복값 확인 (거래일 + 건물명 + 면적 + 층 + 가격)
duplication = df[df.duplicated(subset=[
    'CTRT_DAY',  # 계약일
    'BLDG_NM',   # 건물명
    'ARCH_AREA', # 건물면적(㎡)
    # 'LAND_AREA', # 토지면적(㎡)
    'FLR',       # 층
    'THING_AMT'  # 물건금액(만원)
], keep='first')]

print(f"중복 건수: {len(duplication)}건")
display(duplication.head())

In [ ]:
# # 컬럼 정보
# display(df.info())

# # 상위 5개 행 확인
display(df.head())

# # 결측치 확인
# display(df.isna().sum()) # 전체 확인

# # 요약 통계
# display(df.describe())

# df['price_per_sqm'].describe()

In [ ]:
# 전처리 및 파생변수 생성

# 날짜 변환
df['CTRT_DAY'] = pd.to_datetime(df['CTRT_DAY'], format='%Y%m%d')

# 년월 컬럼 생성
df['year_month'] = df['CTRT_DAY'].dt.to_period('M').astype(str)
# df['year_month'] = df['CTRT_DAY'].dt.to_period('M')

# ㎡당 가격 (만원)
df['price_per_sqm'] = df['THING_AMT'] / df['ARCH_AREA']


Q1 = df['price_per_sqm'].quantile(0.25)
Q3 = df['price_per_sqm'].quantile(0.75)
IQR = Q3 - Q1

outliers = df[
    (df['price_per_sqm'] < Q1 - 1.5*IQR) | 
    (df['price_per_sqm'] > Q3 + 1.5*IQR)
]

print(f"전체: {len(df):,}건")
print(f"이상치: {len(outliers):,}건")
print(f"비율: {len(outliers)/len(df)*100:.1f}%")

# 1. 래미안 원베일리: 98건 (평균 57억)
# 2. 아크로리버파크: 56건 (평균 58억)
# 3. PH129: 2건 (평균 172억!)
# 4. 한양1차: 20건 (평균 ㎡당 7,104만원)

# → 전부 실제 거래!
# → 데이터 오류 아님!
# 제거하면 시장 왜곡

# 규모 구분
def size_group(area):
    if area <= 60: # 18.15평
        return '소형'
    elif area <= 85: # 25.7평
        return '중형'
    else:
        return '대형'

df['size_group'] = df['ARCH_AREA'].apply(size_group)

df.head()


In [ ]:
# 구별 월 평균(핵심 지표)

monthly_gu = (
    df.groupby(['year_month', 'CGG_NM'])
      .agg(
          avg_price_per_sqm=('price_per_sqm', 'mean'), # ㎡당가격
          median_price_per_sqm=('price_per_sqm', 'median'), # 이상치 처리
          avg_price=('THING_AMT', 'mean'),             # 총가격
          txn_cnt=('THING_AMT', 'count')               # 거래량
      )
      .reset_index()
)

monthly_gu.head()

In [ ]:
# from supabase import create_client, Client
# from dotenv import load_dotenv
# import os
# import pandas as pd
# import numpy as np

# load_dotenv()

# SUPABASE_URL = os.getenv("SUPABASE_URL")
# SUPABASE_KEY = os.getenv("SUPABASE_KEY")
# supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# # CSV 읽기 (dtype 경고 해결)
# df = pd.read_csv(
#     r'C:\Users\Administrator\workspace\budongsan3_project\jmj\projects\서울시 부동산 실거래가 정보 2025.csv',
#     low_memory=False
# )

# print(f"원본 데이터: {df.shape}")

# # 컬럼명을 영문으로 변경
# df.columns = [
#     'year', 'gu_code', 'gu_name', 'dong_code', 'dong_name',
#     'jibun_type', 'jibun_type_name', 'bonbun', 'bubun', 'building_name',
#     'contract_date', 'price_10k', 'building_area', 'land_area', 'floor_num',
#     'right_type', 'cancel_date', 'build_year', 'building_usage',
#     'report_type', 'broker_region'
# ]

# # 데이터 클리닝: 공백 문자열과 빈 문자열을 None으로 변환
# for col in df.columns:
#     if df[col].dtype == 'object':  # 문자열 컬럼만
#         df[col] = df[col].apply(lambda x: None if isinstance(x, str) and x.strip() == '' else x)

# # 숫자 컬럼들의 공백 처리
# numeric_cols = ['bonbun', 'bubun', 'jibun_type', 'price_10k', 'building_area', 
#                 'land_area', 'floor_num', 'cancel_date', 'build_year']

# for col in numeric_cols:
#     if col in df.columns:
#         # 공백 문자열을 NaN으로 변환
#         df[col] = pd.to_numeric(df[col], errors='coerce')

# # NaN과 inf를 None으로 변환
# df = df.replace([np.nan, np.inf, -np.inf], None)

# # 딕셔너리 리스트로 변환
# data_to_insert = df.to_dict('records')

# print(f"총 {len(data_to_insert)}개 데이터 업로드 시작...")

# # 1000개씩 배치 업로드
# batch_size = 1000
# success_count = 0
# error_batches = []

# for i in range(0, len(data_to_insert), batch_size):
#     batch = data_to_insert[i:i+batch_size]
#     try:
#         response = supabase.table('all_data').insert(batch).execute()
#         success_count += len(batch)
#         print(f"{success_count}/{len(data_to_insert)} 완료")
#     except Exception as e:
#         print(f"❌ 에러 발생 at {i}: {e}")
#         error_batches.append(i)
        
#         # 에러 발생한 배치의 샘플 데이터 출력
#         print(f"문제 데이터 샘플 (인덱스 {i}):")
#         for j in range(min(3, len(batch))):
#             print(f"  [{i+j}]: {batch[j]}")
        
#         # 계속 진행할지 물어보기
#         continue_upload = input("계속 진행하시겠습니까? (y/n): ")
#         if continue_upload.lower() != 'y':
#             break

# # 최종 확인
# result = supabase.table('all_data').select("*").limit(5).execute()
# print(f"\n✅ 업로드 완료!")
# print(f"성공: {success_count}/{len(data_to_insert)}")
# if error_batches:
#     print(f"에러 발생 배치: {error_batches}")
# print(f"DB 조회: {len(result.data)}개")

# if result.data:
#     display(pd.DataFrame(result.data))

In [ ]:
# 규모별 월 평균 (해석력 높히기)
monthly_size = (
    df.groupby(['year_month', 'CGG_NM', 'size_group'])
      .agg(
          avg_price_per_sqm=('price_per_sqm', 'mean'), # ㎡당가격
          txn_cnt=('THING_AMT', 'count')               # 거래량
      )
      .reset_index()
)

monthly_size.head()

In [ ]:
# 구별 누적 상승률

price_start_end = (
    monthly_gu
    .sort_values('year_month')
    .groupby('CGG_NM')
    .agg(
        start_price=('avg_price_per_sqm', 'first'),  # 시작가격
        end_price=('avg_price_per_sqm', 'last'),     # 종료가격
        start_median=('median_price_per_sqm', 'first'),
        end_median=('median_price_per_sqm', 'last')
    )
)


# 변화량
price_start_end['change_amt'] = (
    price_start_end['end_price'] - price_start_end['start_price']
)

price_start_end['change_pct'] = (
    price_start_end['change_amt'] / price_start_end['start_price'] * 100
)

# 증가율
price_start_end['change_pct_median'] = (
    (price_start_end['end_median'] - price_start_end['start_median']) / 
    price_start_end['start_median'] * 100
)

display(price_start_end)

# change_pct(평균) / change_pct_median(중간값)
display(price_start_end[['change_pct', 'change_pct_median']])

In [ ]:
# 1. 구별 월별 평균 가격 추이
fig1 = px.line(
    monthly_gu,
    x='year_month',
    y='avg_price',
    color='CGG_NM',
    title='강남 3구 월별 평균 실거래가 추이 (2023-2025)',
    labels={
        'year_month': '년월',
        'avg_price': '평균 실거래가 (만원)',
        'CGG_NM': '구'
    },
    markers=True,
    color_discrete_map={
        '강남구': '#FF6B6B',
        '서초구': '#4ECDC4',
        '송파구': '#95E1D3'
    }
)

fig1.update_layout(
    hovermode='x unified',
    xaxis_tickangle=-45,
    height=600
)

fig1.show()

In [ ]:
# 2. 구별 증가율 비교
fig2 = px.bar(
    price_start_end.reset_index(),
    x='CGG_NM',
    y='change_pct',
    title='구별 가격 증가율 (2023-01 대비 2025-12)',
    labels={
        'CGG_NM': '구',
        'change_pct': '증가율 (%)'
    },
    text='change_pct',
    color='change_pct',
    color_continuous_scale='RdYlGn'
)

fig2.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='outside'
)

fig2.update_layout(height=500)
fig2.show()

print('='*80)
print("구별 증가율")
print("="*80)

for gu in price_start_end.index:
    mean_pct = price_start_end.loc[gu, 'change_pct']
    median_pct = price_start_end.loc[gu, 'change_pct_median']
    
    print(f"\n{gu}:")
    print(f"  평균:   {mean_pct:+.1f}%")
    print(f"  중간값: {median_pct:+.1f}%")

In [ ]:
# 3. 규모별 ㎡당 가격 추이
fig3 = px.line(
    monthly_size,
    x='year_month',
    y='avg_price_per_sqm',
    color='size_group',
    facet_col='CGG_NM',  # 구별로 분할
    title='구별·규모별 ㎡당 평균 가격 추이',
    labels={
        'year_month': '년월',
        'avg_price_per_sqm': '㎡당 가격 (만원)',
        'size_group': '규모',
        'CGG_NM': '자치구'
    },
    markers=True
)

fig3.update_xaxes(tickangle=-45)
fig3.update_layout(height=500)
fig3.show()

In [ ]:
# 4. 거래량과 가격 동시 표시 (듀얼 축)
fig4 = go.Figure()

# 강남구 예시
gangnam = monthly_gu[monthly_gu['CGG_NM'] == '강남구']

# 가격 (왼쪽 축)
fig4.add_trace(go.Scatter(
    x=gangnam['year_month'],
    y=gangnam['avg_price'],
    name='평균 가격',
    yaxis='y',
    mode='lines+markers',
    line=dict(color='#FF6B6B', width=3)
))

# 거래량 (오른쪽 축)
fig4.add_trace(go.Bar(
    x=gangnam['year_month'],
    y=gangnam['txn_cnt'],
    name='거래량',
    yaxis='y2',
    opacity=0.3,
    marker_color='#95E1D3'
))

fig4.update_layout(
    title='강남구 가격 vs 거래량 추이',
    xaxis=dict(title='년월', tickangle=-45),
    yaxis=dict(
        title='평균 가격 (만원)',
        side='left'
    ),
    yaxis2=dict(
        title='거래량 (건)',
        side='right',
        overlaying='y'
    ),
    hovermode='x unified',
    height=600
)

fig4.show()

In [ ]:
# 5. 구별 가격 분포 (박스플롯)
fig5 = px.box(
    df,
    x='CGG_NM',
    y='THING_AMT',
    color='size_group',
    title='구별·규모별 가격 분포',
    labels={
        'CGG_NM': '구',
        'THING_AMT': '거래가 (만원)',
        'size_group': '규모'
    },
    points='outliers'  # 이상치만 점으로 표시
)

fig5.update_layout(height=600)
fig5.show()

In [ ]:
# 6. 히트맵 (월별 x 구별 가격)
pivot_data = monthly_gu.pivot(
    index='CGG_NM',
    columns='year_month',
    values='avg_price'
)

fig6 = px.imshow(
    pivot_data,
    title='월별·구별 평균 가격 히트맵',
    labels=dict(x='년월', y='구', color='평균가 (만원)'),
    color_continuous_scale='YlOrRd',
    aspect='auto'
)

fig6.update_xaxes(tickangle=-45)
fig6.update_layout(height=500)
fig6.show()